In [3]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./data/mnist', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [11]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

# tf.layer를 이용한 CNN 모델 

기본적으로 inputs, outputs size, kernel_size 만 넣어주면 활성화 함수와, 컨볼루션 신경망을 만들기 위한 나머지 수치들은 알아서 계산해준다.
특히 Weights 를 계산하는데 xavier_initializer 를 쓰고 있는 등, 크게 신경쓰지 않아도 일반적으로 효율적인 신경망을 만들어준다.

In [12]:
L1 = tf.layers.conv2d(X, 32, [3, 3], activation=tf.nn.relu) # filter_size=32, kernel_size=[3, 3]
L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2]) # pool_size=[2, 2], strides=[2, 2]
L1 = tf.layers.dropout(L1, 0.7, is_training)

L2 = tf.layers.conv2d(L1, 64, [3, 3], activation=tf.nn.relu) 
L2 = tf.layers.max_pooling2d(L2, [2, 2], [2, 2]) 
L2 = tf.layers.dropout(L2, 0.7, is_training)

L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)
L3 = tf.layers.dropout(L3, 0.5, is_training)

model= tf.layers.dense(L3, 10, activation=None)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

# 모델 학습

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys, is_training: True})
        
        total_cost += cost_val
    
    print('Epoch :', '%04d' % (epoch + 1), 'Avg. cost = ', '{:.4f}'.format(total_cost/ total_batch))

Epoch : 0001 Avg. cost =  0.1873
Epoch : 0002 Avg. cost =  0.0533
Epoch : 0003 Avg. cost =  0.0367
Epoch : 0004 Avg. cost =  0.0258
Epoch : 0005 Avg. cost =  0.0211
Epoch : 0006 Avg. cost =  0.0148
Epoch : 0007 Avg. cost =  0.0119
Epoch : 0008 Avg. cost =  0.0098
Epoch : 0009 Avg. cost =  0.0083
Epoch : 0010 Avg. cost =  0.0072
Epoch : 0011 Avg. cost =  0.0062
Epoch : 0012 Avg. cost =  0.0049
Epoch : 0013 Avg. cost =  0.0069
Epoch : 0014 Avg. cost =  0.0027
Epoch : 0015 Avg. cost =  0.0047


# 결과 확인

In [14]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도 :', sess.run(accuracy, feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1), Y: mnist.test.labels, is_training: False}))

정확도 : 0.9923
